In [17]:
import os

In [18]:
%pwd

'd:\\dlproject'

In [19]:
os.chdir("../")

In [20]:
%pwd

'd:\\'

In [34]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [35]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [36]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [37]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [45]:
import py7zr  # Make sure this is at the top of your file
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
    
        except Exception as e:
            raise e

        

    def extract_zip_file(self):
        """
        Extracts the .7z file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with py7zr.SevenZipFile(self.config.local_data_file, mode='r') as archive:
                archive.extractall(path=unzip_path)
                logger.info(f"Extracted .7z file to {unzip_path}")
        except Exception as e:
            logger.error(f"Failed to extract .7z file: {e}")
            raise e


In [46]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-20 08:18:56,787: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-20 08:18:56,793: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-20 08:18:56,794: INFO: common: created directory at: artifacts]
[2025-04-20 08:18:56,798: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-20 08:18:56,800: INFO: 20753405: Downloading data from https://drive.google.com/file/d/1Bg7xMZZ_LgyDZFQVAhY9mEYvcOG4CCMu/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Bg7xMZZ_LgyDZFQVAhY9mEYvcOG4CCMu
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Bg7xMZZ_LgyDZFQVAhY9mEYvcOG4CCMu&confirm=t&uuid=18b7fafc-f324-4e78-b1e6-603760cb8bb7
To: d:\dlproject\artifacts\data_ingestion\data.zip
100%|██████████| 43.3M/43.3M [00:20<00:00, 2.07MB/s]

[2025-04-20 08:20:18,434: INFO: 20753405: Downloaded data from https://drive.google.com/file/d/1Bg7xMZZ_LgyDZFQVAhY9mEYvcOG4CCMu/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


[2025-04-20 08:21:16,748: INFO: 20753405: Extracted .7z file to artifacts/data_ingestion]


In [44]:
%pip install py7zr


                                              0.0/67.9 kB ? eta -:--:--
     ------------------                       30.7/67.9 kB ? eta -:--:--
     ----------------------------           51.2/67.9 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 67.9/67.9 kB 529.0 kB/s eta 0:00:00
                                              0.0/1.8 MB ? eta -:--:--
     --                                       0.1/1.8 MB 5.5 MB/s eta 0:00:01
     --                                       0.1/1.8 MB 1.3 MB/s eta 0:00:02
     -----                                    0.2/1.8 MB 1.7 MB/s eta 0:00:01
     -----                                    0.2/1.8 MB 1.7 MB/s eta 0:00:01
     ---------                                0.4/1.8 MB 1.8 MB/s eta 0:00:01
     ----------                               0.5/1.8 MB 2.1 MB/s eta 0:00:01
     ----------------------                   1.0/1.8 MB 2.9 MB/s eta 0:00:01
     ----------------------                   1.0/1.8 MB 2.6 MB/s eta 0:00:01



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
